In [85]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [120]:
steam = pd.read_csv ('../Datasets/data_Api/function3.csv', usecols=['user_id','item_id','sentiment_analysis','recommend','app_name'])

In [121]:
df = steam
def recomendacion_juego(df, input_item_id, num_recommendations=5):
    # Filtrar el DataFrame para obtener solo las filas relacionadas con el juego de entrada
    input_item = df[df['item_id'] == input_item_id]

    if input_item.empty:
        return "El ID de producto no se encuentra en el conjunto de datos."

    # Extraer el usuario y sus recomendaciones para ese juego
    user_item_matrix = df.pivot_table(index='user_id', columns='item_id', values='recommend', fill_value=0)

    # Calcular la similitud coseno entre el juego de entrada y todos los demás juegos
    similarity_scores = cosine_similarity(user_item_matrix.T)

    # Obtener las puntuaciones de similitud para el juego de entrada
    input_item_scores = similarity_scores[input_item_id]

    # Obtener los índices de los juegos más similares (excluyendo el juego de entrada)
    similar_items_indices = input_item_scores.argsort()[:-1][-num_recommendations:]

    # Obtener los IDs y nombres de los juegos más similares
    similar_items = user_item_matrix.columns[similar_items_indices]

    # Obtener nombres de juegos correspondientes a los IDs
    game_names_df = df[df['item_id'].isin(similar_items)][['item_id', 'app_name']]

# Fusionar los DataFrames para obtener nombres correspondientes a los IDs
    merged_df = pd.merge(pd.DataFrame(similar_items, columns=['item_id']), game_names_df, on='item_id')

# Imprimir las recomendaciones de juegos con saltos de línea y números de índice
    print(f"Juegos recomendados similares al juego con ID {input_item_id}:")

    for idx, (item_id, game_name) in enumerate(zip(merged_df['item_id'], merged_df['app_name']), start=1):
        print(f"{idx}. Juego ID: {item_id}, Name: {game_name}")
        

# Ejemplo de uso:
input_item_id = 80
recomendacion_juego(df, input_item_id)

Juegos recomendados similares al juego con ID 80:
1. Juego ID: 40970, Name: Stronghold Crusader HD
2. Juego ID: 40970, Name: Stronghold Crusader HD
3. Juego ID: 40970, Name: Stronghold Crusader HD
4. Juego ID: 40970, Name: Stronghold Crusader HD
5. Juego ID: 40970, Name: Stronghold Crusader HD
6. Juego ID: 232890, Name: Stronghold Crusader 2
7. Juego ID: 232890, Name: Stronghold Crusader 2
8. Juego ID: 232890, Name: Stronghold Crusader 2
9. Juego ID: 232890, Name: Stronghold Crusader 2
10. Juego ID: 209270, Name: Hero Academy
11. Juego ID: 209270, Name: Hero Academy
12. Juego ID: 209270, Name: Hero Academy
13. Juego ID: 330100, Name: BossConstructor
14. Juego ID: 342560, Name: Airships: Conquer the Skies


In [43]:
steam[steam['item_id'] == 214950]

,app_name,user_id,item_id,recommend,sentiment_analysis
27777,Total War™: ROME II - Emperor Edition,76561198057086148,214950,1,1


Si es un sistema de recomendación user-item:

def recomendacion_usuario( id de usuario ): Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.

In [123]:
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import KNNBasic

# Crear un objeto Reader
reader = Reader(rating_scale=(0, 1))

# Cargar el DataFrame en el formato correcto para Surprise
data = Dataset.load_from_df(df[['user_id', 'item_id', 'recommend']], reader)

# Dividir el conjunto de datos en entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Configurar el algoritmo de filtrado colaborativo basado en usuarios
sim_options = {
    'name': 'cosine',
    'user_based': True,
}
algo = KNNBasic(sim_options=sim_options)

# Entrenar el modelo
algo.fit(trainset)

# Obtener las recomendaciones para el usuario de entrada
user_id = 'EizanAratoFujimaki'
user_items = df[df['user_id'] == user_id][['item_id', 'recommend']].values.tolist()

# Predecir las recomendaciones para el usuario
predictions = [algo.predict(user_id, item_id) for item_id, _ in user_items]

# Filtrar las recomendaciones con una puntuación alta
filtered_recommendations = [(prediction.iid, prediction.est) for prediction in predictions if prediction.est > 0.5]

# Ordenar las recomendaciones por puntuación descendente
sorted_recommendations = sorted(filtered_recommendations, key=lambda x: x[1], reverse=True)

print("Juegos recomendados para el usuario con ID", user_id, ":", sorted_recommendations[:num_recommendations])



Juegos recomendados para el usuario con ID EizanAratoFujimaki : [('-Ultrix', 'Half-Life'), ('76561198020928326', 'Half-Life'), ('76561198125270928', 'Half-Life'), ('coolman1342', 'Half-Life'), ('h00py', 'Half-Life')]
